# Eating Detection Exploration Pipeline

The code below executes the eating detection pipeline. There are three major sections

- The Data Preparation pipeline is responsible for loading the data and spliting to different leave one person out data set. 
- We then benchmark different classical models
- Finally, we benchmark some DNN approaches. 

In [ ]:
import json
import time
from datetime import datetime
import numpy as np
import pandas as pd
import pickle
import glob
import os
from packaging import version
from time import sleep, strftime

# Plotting
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters

from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive',force_remount=True)
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score
import sklearn.model_selection
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis


import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
import math

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Fri Dec 10 03:31:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

The features directory has a file structured so every column represents features for 5 seconds window. The first row has the window number and the label of that window the rest of the 43 rows are features. These windows are consecutive in time with a time difference of 1 second between every adjacent windows (so it's a 5-sec sliding window sliding by 1 second at a time)

5 Gyros, 5*3 = 15 channels + 1 proximity

row values divided by 16, each ordered as below:

Median
Entropy
Variance
RMS
zero cross - Variance :
zero crossing
Select the below features from the gyroscopes & proximi values [7,8,9,10,11,12,16] ==> [7,8,9] --> G1, [10,11,12] --> G2 and 16 --> prox

6 * 7 channels == xg1,yg1,zg1,xg2,yg2,zg2,Proximity.

File: 5_slid_1_featuerslabels_DStest2.csv

Look for all the activity folder(5) in segmented data, and in each activity leave on participant out.

Labels: remove None, change label with _'s to only the first values.

In [ ]:
activities=['snack','meeting','TV','hike','gym']
classes=['silent','eating','drinking','talking','walking']
features_list=['Median','Entropy','Variance','RMS','ZeroCrossingVar',
               'ZeroCrossing']
all_sensor_list = ['g1','g2','g3','g4','g5','prox']
sub_sensor_list = ['g3','g4','prox']
sensors={'g1':np.array([0,1,2]),'g2':np.array([3,4,5]),'g3':np.array([6,7,8]),'g4':np.array([9,10,11]),'g5':np.array([12,13,14]),'prox':np.array([15])}


# Creates {'Median': 1, 'Entropy': 17, 'Variance': 33, 'RMS': 49, 'ZeroCrossingVar': 65, 'ZeroCrossing': 81}
index=0
feature_index={key: 0 for key in features_list}
for feature in features_list:
    count= 16
    # feature_index[feature]=index+1 #for numpy array
    feature_index[feature]=index # for Pandas
    index+=count
    
participants_all = []
participants_include = []
participant_exclude = []


# Paths!
data_directory = 'dataset/analysis/data/SegmentedData'
lopo_data_directory = 'dataset/samples/lopo_data/'
features_directory = 'dataset/data_features/'
models_directory = 'dataset/models/'
models_directory_lopo = 'dataset/lopo-models/'
results_directory_lopo = 'dataset/results_lopo/'


# # Dataframes
global entire_data, entire_test_data, entire_train_data
entire_data = pd.DataFrame()
entire_test_data = pd.DataFrame()
entire_train_data = pd.DataFrame()


In [ ]:
indexToSelect = []
for sensor in sub_sensor_list:
  for feature in features_list:
    # print(list(sensors[sensor]+feature_index[feature]))
    for itm in list(sensors[sensor]+feature_index[feature]):
      indexToSelect.append(itm)
indexToSelect.sort()
print (indexToSelect)

# Data Pre-Preparation

In [ ]:
def create_features_file_for_all_activities(csv_file, path, new_file_name):
  global entire_data
  csv_data = pd.read_csv(path+'/'+csv_file)
  # print (csv_data.head())
  filtered_csv_data = csv_data.filter(indexToSelect,axis=0).reset_index(drop=True)
  print(filtered_csv_data.shape, csv_data.shape, csv_data.index)
  # print(filtered_csv_data.head(),'\n',filtered_csv_data.tail())
  filtered_csv_data.to_csv(features_directory+new_file_name,index=False)

  filtered_csv_data.columns = filtered_csv_data.columns.str.split(': ').str[1]
  filtered_csv_data = filtered_csv_data.T
  print(filtered_csv_data.shape)

  if entire_data.empty:
    print('DataFrame is empty!')
    entire_data = filtered_csv_data.copy(deep=True)
  else:
    entire_data = entire_data.append(filtered_csv_data,ignore_index=True)

def clear_entire_data():
  global entire_data
  del(entire_data)
  entire_data = pd.DataFrame()

In [ ]:
def create_lopo_test_data_features(csv_file, path, new_file_name):
  global entire_test_data
  csv_data = pd.read_csv(path+'/'+csv_file)
  filtered_csv_data = csv_data.filter(indexToSelect,axis=0).reset_index(drop=True)
  print(filtered_csv_data.shape, csv_data.shape, csv_data.index)
  filtered_csv_data.columns = filtered_csv_data.columns.str.split(': ').str[1]
  filtered_csv_data = filtered_csv_data.T
  print(filtered_csv_data.shape)

  if entire_test_data.empty:
    print('DataFrame is empty!')
    entire_test_data = filtered_csv_data.copy(deep=True)
  else:
    entire_test_data = entire_test_data.append(filtered_csv_data)


def create_lopo_train_data_features(csv_file, path, new_file_name):
  global entire_train_data
  csv_data = pd.read_csv(path+'/'+csv_file)
  filtered_csv_data = csv_data.filter(indexToSelect,axis=0).reset_index(drop=True)
  print(filtered_csv_data.shape, csv_data.shape, csv_data.index)
  filtered_csv_data.columns = filtered_csv_data.columns.str.split(': ').str[1]
  filtered_csv_data = filtered_csv_data.T

  if entire_train_data.empty:
    print('DataFrame is empty!')
    entire_train_data = filtered_csv_data.copy(deep=True)
  else:
    entire_train_data = entire_train_data.append(filtered_csv_data)
  
def clear_entire_data_lopo():
  global entire_train_data, entire_test_data
  del(entire_train_data)
  del(entire_test_data)

  entire_train_data = pd.DataFrame()
  entire_test_data = pd.DataFrame()

In [ ]:
def update_data_to_five_classes():
  global entire_train_data
  global entire_test_data
  entire_train_data = entire_train_data.drop('none')
  entire_train_data = entire_train_data.rename(index={'drinking_walking': 'drinking'})
  entire_train_data = entire_train_data.rename(index={'walking_talking': 'walking'})
  entire_train_data = entire_train_data.rename(index={'drinking_talking': 'drinking'})
  entire_train_data = entire_train_data.rename(index={'eating_talking': 'eating'})
  entire_train_data = entire_train_data.rename(index={'eating_walking': 'eating'})
  entire_train_data = entire_train_data.rename(index={'eating_walking_talking': 'eating'})
  entire_train_data = entire_train_data.rename(index={'drinking_walking_talking': 'drinking'})
  entire_train_data = entire_train_data.rename(index={'eating_drinking': 'eating'})
  entire_train_data = entire_train_data.rename(index={'eating_drinking_talking': 'eating'})

  entire_test_data = entire_test_data.drop('none')
  entire_test_data = entire_test_data.rename(index={'drinking_walking': 'drinking'})
  entire_test_data = entire_test_data.rename(index={'walking_talking': 'walking'})
  entire_test_data = entire_test_data.rename(index={'drinking_talking': 'drinking'})
  entire_test_data = entire_test_data.rename(index={'eating_talking': 'eating'})
  entire_test_data = entire_test_data.rename(index={'eating_walking': 'eating'})
  entire_test_data = entire_test_data.rename(index={'eating_walking_talking': 'eating'})
  entire_test_data = entire_test_data.rename(index={'drinking_walking_talking': 'drinking'})
  entire_test_data = entire_test_data.rename(index={'eating_drinking': 'eating'})
  entire_test_data = entire_test_data.rename(index={'eating_drinking_talking': 'eating'})
  print("updating data to five classes")

The below code loops throught the dataset and creates a Leave-One-Out dataset.

In [ ]:
ext = "*.csv"
all_csv_files =[]

if not entire_train_data.empty:
    clear_entire_data_lopo()

# Find all participants
for path, subdir, files in os.walk(data_directory):
    # print (path, subdir) 
    for file in files:
       if file =='5_slid_1_featuerslabels_DStest2.csv':
         participant_name = str(path.split(os.path.sep)[-1])
         if participant_name not in participants_all:
           participants_all.append(participant_name)
          #  print(participants_all)

print(participants_all)

flag =0
# Create LOPO/LOOCV dataset for the participants. 
for participant in participants_all:
  
  lopo_path = lopo_data_directory + 'leave_participant_' + participant 
  
  print('Leaving Path', participant)
  print('Leaving Path', lopo_path)

  if not os.path.exists(lopo_path):
    # Create a new directory because it does not exist 
    os.makedirs(lopo_path)
    print("The new directory is created!",lopo_path)

  for path, subdir, files in os.walk(data_directory):
      print (path, subdir) 
      for file in files:
        if file =='5_slid_1_featuerslabels_DStest2.csv':
          participant_folder = str(path.split(os.path.sep)[-1])
          new_file_name = str(path.split(os.path.sep)[-2])+'_'+ \
                str(path.split(os.path.sep)[-1])+\
                '_filtered_5_slid_1_featuerslabels_DStest2.csv'
          print(new_file_name)
          # testing:
          if participant == participant_folder:
            create_lopo_test_data_features(file, path, new_file_name)
          else:
            create_lopo_train_data_features(file, path, new_file_name)

  update_data_to_five_classes()

  print('\n\n\n********** Final shape entire_train_data')
  print(entire_train_data.shape, entire_train_data.index)
  print('saving ' + lopo_path+'/train_data.csv' )
  # save test and train data
  entire_train_data.to_csv(lopo_path+'/train_data.csv')

  print('\n\n\n********** Final shape entire_test_data')
  print(entire_test_data.shape, entire_test_data.index)
  print('saving ' + lopo_path+'/test_data.csv' )
  entire_test_data.to_csv(lopo_path+'/test_data.csv')
  clear_entire_data_lopo()
  # break;

# Load the Data

In [ ]:
participant = ''
data_directory_train = lopo_data_directory+'leave_participant_'+participant+'/train_data.csv'
data_train = pd.read_csv(data_directory_train,index_col=0)
data_train.head()

sns.countplot(x = entire_train_data.index,
              data = entire_train_data,
              order = entire_train_data.index.value_counts().index);

In [ ]:
data_directory_test = '/content/drive/MyDrive/TFLite-Exploration-Eating/Samples/lopo_data/leave_participant_S/test_data.csv'
entire_data_t = pd.read_csv(data_directory_test,index_col=0)
entire_data_t.shape

# Loading Classical models

 Leave one out data logging with prediction logging.

In [ ]:
participants_all = []
# Training Basic Models.
models_list = [knn_model, svm_rbf_model, svc_model, dtc_model, rnf_model,
               mlp_model, ada_model, gnb_model]


results_file_csv = results_directory_lopo + 'results_lopo.csv'
results = []

if not os.path.exists(results_directory_lopo):
  print("make-dir", results_directory_lopo)
  os.makedirs(results_directory_lopo)
               
for participant in participants_all:
  print('leave_participant_',participant)

  data_directory_train = lopo_data_directory+'leave_participant_'+participant+'/train_data.csv'
  data_directory_test = lopo_data_directory+'leave_participant_'+participant+'/test_data.csv'
  
  print('reading',data_directory_train)
  print('reading',data_directory_test)

  data_train = pd.read_csv(data_directory_train,index_col=0)
  data_test = pd.read_csv(data_directory_test,index_col=0)

  X_train = data_train.to_numpy()
  y_train = data_train.index.to_numpy()
  X_test = data_test.to_numpy()
  y_test = data_test.index.to_numpy()

  ##########################
  #######Training
  ##########################
  print('\n\n###################')
  print("All Training Started", 'leave_participant_', participant)
  print('###################\n')
  for model in models_list:
    print("Training Started", 'leave_participant_', participant , model)
    start = time.time()
    print(model)
    model.fit(X_train, y_train)
    end = time.time()
    print("Time taken to train: ", str(end - start))

    print("Storing the Models")

    model_name = type(model).__name__
    location = models_directory_lopo+ model_name+ '/'
    if not os.path.exists(location):
      print("make-dir", location)
      os.makedirs(location)
    location_filename = location + 'leave_participant_'+participant+'_'+model_name +".pickle" 
    pickle.dump(model, open(location_filename, 'wb'))
    # print("Done Storing the Models")
    print("Training Ended", 'leave_participant_', participant , model)
  print("All Training Done", 'leave_participant_', participant)
  
  ##########################
  #######Prediction
  ##########################
  print('\n###################')
  print("All Prediction Started", 'leave_participant_', participant)
  print('###################\n')
  for model in models_list:
    print("Prediction Started", 'leave_participant_', participant , type(model).__name__)
    start = time.time()
    print(model)
    y_pred = model.predict(X_test)
    end = time.time()
    prediction_time = str(end - start)
    accuracy = accuracy_score(y_test, y_pred)
    print("Time taken to predict: ", prediction_time)
    print("Accuracy: ", accuracy)      
    with open(results_file_csv, "a") as log:
      log.write("{0},{1},leave_participant_{2},{3},{4}\n".format(strftime("%Y-%m-%d %H:%M:%S"),type(model).__name__,participant,str(accuracy),str(prediction_time)))
    print('wrote_to_results')
    results.append([type(model).__name__,participant,str(accuracy),str(prediction_time)])
    print([[x,y_pred.tolist().count(x)] for x in set(y_pred.tolist())])
    print("Prediction Done", 'leave_participant_', participant)
    print("\n\n")
  print("All Prediction Ended", 'leave_participant_', participant)

# Deep Learning Model Explorations

## Model 1 : Custom DNN

In [ ]:
# Train TF model - 1 Definition

# Build the model and train it
model = tf.keras.Sequential()

model.add(tf.keras.layers.Dense(50, activation='relu')) # relu is used for performance
model.add(tf.keras.layers.Dense(15, activation='relu'))
model.add(tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')) # softmax is used, because we only expect one activity to occur per input


opt = tf.keras.optimizers.Adam(learning_rate=4e-4)


In [ ]:
for participant in participants_all:
  print('leave_participant_',participant)

  data_directory_train = lopo_data_directory+'leave_participant_'+participant+'/train_data.csv'
  data_directory_test = lopo_data_directory+'leave_participant_'+participant+'/test_data.csv'
  
  print('reading',data_directory_train)
  print('reading',data_directory_test)

  data_train = pd.read_csv(data_directory_train,index_col=0)
  data_test = pd.read_csv(data_directory_test,index_col=0)

  X_train = data_train.to_numpy()
  y_train = data_train.index.to_numpy()
  X_test = data_test.to_numpy()
  y_test = data_test.index.to_numpy()

  enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

  y_train_enc = np.array(y_train).reshape(-1, 1)
  y_test_enc = np.array(y_test).reshape(-1, 1) 
  y_val_enc = np.array(y_test_val).reshape(-1, 1) 

  enc = enc.fit(y_train_enc)

  y_train_enc = enc.transform(y_train_enc)
  y_test_enc = enc.transform(y_test_enc)
  y_val_enc = enc.transform(y_val_enc)

  print("Training Started", 'leave_participant_', participant , model)

  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  history = model.fit(X_train, y_train_enc, validation_data=(X_test_val,y_val_enc), epochs=250, batch_size=1)

  plt.plot(history.history['loss'], label='train')
  plt.plot(history.history['val_loss'], label='test')
  plt.legend();
  
  location=models_directory+ 'five-classes-new-type-input-all-data/' + 'TF-model-1' + '/'
  if not os.path.exists(location):
    print("make-dir", location)
    os.makedirs(location)
    model.save(location+'leave_participant_'+participant+'_custom')
    if not os.path.exists(location):
      print("make-dir", location)
      os.makedirs(location)
      # print("Done Storing the Models")
    print("Training Ended", 'leave_participant_', participant , model)

# Model 2 - TF

In [ ]:
model_2 = tf.keras.Sequential([
            layers.Dense(128, activation='relu'),
            layers.Dense(128, activation='relu'),
            layers.Dropout(.1),
            layers.Dense(1)])


In [ ]:
 for participant in participants_all:
  print('leave_participant_',participant)

  data_directory_train = lopo_data_directory+'leave_participant_'+participant+'/train_data.csv'
  data_directory_test = lopo_data_directory+'leave_participant_'+participant+'/test_data.csv'
  
  print('reading',data_directory_train)
  print('reading',data_directory_test)

  data_train = pd.read_csv(data_directory_train,index_col=0)
  data_test = pd.read_csv(data_directory_test,index_col=0)

  X_train = data_train.to_numpy()
  y_train = data_train.index.to_numpy()
  X_test = data_test.to_numpy()
  y_test = data_test.index.to_numpy()

  enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

  y_train_enc = np.array(y_train).reshape(-1, 1)
  y_test_enc = np.array(y_test).reshape(-1, 1) 
  y_val_enc = np.array(y_test_val).reshape(-1, 1) 

  enc = enc.fit(y_train_enc)

  y_train_enc = enc.transform(y_train_enc)
  y_test_enc = enc.transform(y_test_enc)
  y_val_enc = enc.transform(y_val_enc)

  print("Training Started", 'leave_participant_', participant , model)
  
  model_2.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])
  
  history = model_2.fit(X_train, y_train_enc, validation_data=(X_test_val,y_val_enc), epochs=250, batch_size=1)


  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  history = model.fit(X_train, y_train_enc, validation_data=(X_test_val,y_val_enc), epochs=250, batch_size=1)

  plt.plot(history.history['loss'], label='train')
  plt.plot(history.history['val_loss'], label='test')
  plt.legend();
  
  location=models_directory+ 'five-classes-new-type-input-all-data/' + 'TF-model-2' + '/'
  if not os.path.exists(location):
    print("make-dir", location)
    os.makedirs(location)
    model.save(location+'leave_participant_'+participant+'_custom')
    if not os.path.exists(location):
      print("make-dir", location)
      os.makedirs(location)
      # print("Done Storing the Models")
    print("Training Ended", 'leave_participant_', participant , model)

# Model 3 - GBT

In [ ]:
# Install TensorFlow Dececision Forests.
!pip install tensorflow_decision_forests

# Use wurlitzer to capture training logs.
!pip install wurlitzer

In [ ]:
import tensorflow_decision_forests as tfdf

try:
  from wurlitzer import sys_pipes
except:
  from colabtools.googlelog import CaptureLog as sys_pipes

from IPython.core.magic import register_line_magic
from IPython.display import Javascript

# Check the version of TensorFlow Decision Forests
print("Found TensorFlow Decision Forests v" + tfdf.__version__)

In [ ]:
participants_all = []

results_file_csv = results_directory_lopo + 'results_lopo.csv'
results = []

if not os.path.exists(results_directory_lopo):
  print("make-dir", results_directory_lopo)
  os.makedirs(results_directory_lopo)
               
for participant in participants_all:
  print('leave_participant_',participant)

  data_directory_train = lopo_data_directory+'leave_participant_'+participant+'/train_data.csv'
  data_directory_test = lopo_data_directory+'leave_participant_'+participant+'/test_data.csv'
  
  print('reading',data_directory_train)
  print('reading',data_directory_test)

  data_train = pd.read_csv(data_directory_train,index_col=0)
  data_test = pd.read_csv(data_directory_test,index_col=0)
  
  X_train = data_train.to_numpy()
  y_train = data_train.index.to_numpy()
  X_test = data_test.to_numpy()
  y_test = data_test.index.to_numpy()

  print(X_train.shape)
  print(X_test.shape)
  print(y_train.shape)
  print(y_test.shape)
  break;

  train_ds_pd = data_train.copy(deep=True)
  train_ds_pd['label'] = data_train.index
  train_ds_pd['label'].unique()
  test_ds_pd = data_test.copy(deep=True)
  test_ds_pd['label'] = data_test.index

  train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label='label')
  test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label='label')

  for feature_batch, label_batch in test_ds.take(1):
    print('Every feature:', list(feature_batch.keys()))
    print('A batch of inputs:', feature_batch)
    print('A batch of targets:', label_batch )
  type(test_ds)

  ds = test_ds.take(1)  # Only take a single example

  for example in ds:  # example is `{'image': tf.Tensor, 'label': tf.Tensor}`
    print(list(example[0])
    image = example["image"]
    label = example["label"]
    print(image.shape, label)
  
  model_1 = tfdf.keras.GradientBoostedTreesModel(
    num_trees=500,
    growing_strategy="BEST_FIRST_GLOBAL",
    max_depth=8,
    split_axis="SPARSE_OBLIQUE",
    categorical_algorithm="RANDOM",
    )

  # Optionally, add evaluation metrics.
  model_1.compile(
      metrics=["accuracy"])

  # Train the model.
  # "sys_pipes" is optional. It enables the display of the training logs.
  with sys_pipes():
    model_1.fit(x=train_ds)
  
  evaluation = model_1.evaluate(test_ds, return_dict=True)

  location=models_directory+ 'five-classes-new-type-input-all-data/' + 'GBT-TF-model' + '/'
  if not os.path.exists(location):
    print("make-dir", location)
    os.makedirs(location)
    model.save(location+'leave_participant_'+participant+'_custom')
    if not os.path.exists(location):
      print("make-dir", location)
      os.makedirs(location)
      # print("Done Storing the Models")
    print("Training Ended", 'leave_participant_', participant , model)

# Model 4 -  DNN Model 

In [ ]:
model_m = Sequential()
model_m.add(Reshape((TIME_PERIODS, 3), input_shape=(input_shape,)))
model_m.add(Dense(100, activation='relu'))
model_m.add(Dense(100, activation='relu'))
model_m.add(Dense(100, activation='relu'))
model_m.add(Flatten())
model_m.add(Dense(num_classes, activation='softmax'))
print(model_m.summary())

In [ ]:
 for participant in participants_all:
  print('leave_participant_',participant)

  data_directory_train = lopo_data_directory+'leave_participant_'+participant+'/train_data.csv'
  data_directory_test = lopo_data_directory+'leave_participant_'+participant+'/test_data.csv'
  
  print('reading',data_directory_train)
  print('reading',data_directory_test)

  data_train = pd.read_csv(data_directory_train,index_col=0)
  data_test = pd.read_csv(data_directory_test,index_col=0)

  X_train = data_train.to_numpy()
  y_train = data_train.index.to_numpy()
  X_test = data_test.to_numpy()
  y_test = data_test.index.to_numpy()

  enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

  y_train_enc = np.array(y_train).reshape(-1, 1)
  y_test_enc = np.array(y_test).reshape(-1, 1) 
  y_val_enc = np.array(y_test_val).reshape(-1, 1) 

  enc = enc.fit(y_train_enc)

  y_train_enc = enc.transform(y_train_enc)
  y_test_enc = enc.transform(y_test_enc)
  y_val_enc = enc.transform(y_val_enc)

  print("Training Started", 'leave_participant_', participant , model)
  
  model_2.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])
  
  history = model_2.fit(X_train, y_train_enc, validation_data=(X_test_val,y_val_enc), epochs=250, batch_size=1)


  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  history = model.fit(X_train, y_train_enc, validation_data=(X_test_val,y_val_enc), epochs=250, batch_size=1)

  plt.plot(history.history['loss'], label='train')
  plt.plot(history.history['val_loss'], label='test')
  plt.legend();
  
  location=models_directory+ 'five-classes-new-type-input-all-data/' + 'TF-model-4' + '/'
  if not os.path.exists(location):
    print("make-dir", location)
    os.makedirs(location)
    model.save(location+'leave_participant_'+participant+'_custom')
    if not os.path.exists(location):
      print("make-dir", location)
      os.makedirs(location)
      # print("Done Storing the Models")
    print("Training Ended", 'leave_participant_', participant , model)